In diesem Skript wird nicht auf ein sequentielles Model (Sequential) gesetzt, sondern auf ein Model

In [4]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPool2D

from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from mnistData import MNIST

In [5]:
np.random.seed(0)
tf.random.set_seed(0)

In [6]:
def build_model(
    filters_1: int,
    kernel_size_1: int,
    filters_2: int,
    kernel_size_2: int,
    filters_3: int,
    kernel_size_3: int
) -> Model:
    input_img = Input(shape=(28,28,1))

    x = Conv2D(filters=filters_1, kernel_size=kernel_size_1, padding='same')(input_img)
    x = Activation("relu")(x)
    x = Conv2D(filters=filters_1, kernel_size=kernel_size_1, padding='same')(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)

    x = Conv2D(filters=filters_2, kernel_size=kernel_size_2, padding='same')(input_img)
    x = Activation('relu')(x)
    x = Conv2D(filters=filters_2, kernel_size=kernel_size_2, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPool2D()(x)

    x = Conv2D(filters=filters_3, kernel_size=kernel_size_3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filters_3, kernel_size=kernel_size_3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPool2D()(x)

    x = Flatten()(x)
    x = Dense(units=128)(x)
    x = Activation("relu")(x)
    x = Dense(units=10)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )

    model.compile( 
        loss='categorical_crossentropy',
        optimizer='Adam',
        metrics=['accuracy']
    )
    
    model.summary()

    return model

In [7]:
data = MNIST(with_normalization=True)
x_train, y_train = data.get_train_set()

param_distributions = {
    # Bei einer Random Search gibt man eine Zufallsverteilung an in der gesamplet wird
    'filters_1': randint(8, 64),
    'kernel_size_1': randint(3, 8),
    'filters_2': randint(8, 64),
    'kernel_size_2': randint(3, 8),
    'filters_3': randint(8, 64),
    'kernel_size_3': randint(3, 8)
}

keras_clf = KerasClassifier(
    build_fn=build_model, # Funktionsobjekt übergeben, Fit-Methode ist hier intern gelöst und Parameter müssen angegeben werden
    epochs=3,
    batch_size=128,
    verbose=1
)

rand_cv = RandomizedSearchCV(
    estimator=keras_clf,
    param_distributions=param_distributions,
    n_iter=10,
    n_jobs=1,
    verbose=0,
    cv=3
)

rand_result = rand_cv.fit(
    X=x_train,
    y=y_train
)

C:\Users\gutsc\AppData\Local\Temp/ipykernel_27460/2244875532.py:14: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_clf = KerasClassifier(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 55)        2035      
                                                                 
 activation_2 (Activation)   (None, 28, 28, 55)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 55)        108955    
                                                                 
 activation_3 (Activation)   (None, 28, 28, 55)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 55)       0         
 2D)                                                             
                                                             

In [8]:
print(f"Best: {rand_result.best_score_} using {rand_result.best_params_}")
means = rand_result.cv_results_["mean_test_score"]
stds = rand_result.cv_results_["std_test_score"]
params = rand_result.cv_results_["params"]

for mean, std, param in zip(means, stds, params):
    print(f"Acc: {mean} (+/- {std * 2}) with: {param}")

Best: 0.9882000088691711 using {'filters_1': 52, 'filters_2': 55, 'filters_3': 61, 'kernel_size_1': 3, 'kernel_size_2': 6, 'kernel_size_3': 6}
Acc: 0.9882000088691711 (+/- 0.002673322709131638) with: {'filters_1': 52, 'filters_2': 55, 'filters_3': 61, 'kernel_size_1': 3, 'kernel_size_2': 6, 'kernel_size_3': 6}
Acc: 0.984933336575826 (+/- 0.004619037524970768) with: {'filters_1': 11, 'filters_2': 47, 'filters_3': 17, 'kernel_size_1': 6, 'kernel_size_2': 5, 'kernel_size_3': 7}
Acc: 0.9870666662851969 (+/- 0.0014055205434198041) with: {'filters_1': 31, 'filters_2': 14, 'filters_3': 32, 'kernel_size_1': 3, 'kernel_size_2': 7, 'kernel_size_3': 5}
Acc: 0.9874000151952108 (+/- 0.0013366405451055133) with: {'filters_1': 9, 'filters_2': 46, 'filters_3': 47, 'kernel_size_1': 3, 'kernel_size_2': 4, 'kernel_size_3': 4}
Acc: 0.9821999867757162 (+/- 0.001979886341772047) with: {'filters_1': 21, 'filters_2': 16, 'filters_3': 17, 'kernel_size_1': 7, 'kernel_size_2': 6, 'kernel_size_3': 3}
Acc: 0.98484